In [2]:
import sys
sys.path.append("../")
import time

import torch
import torch.nn.functional as F
import numpy as np

from utils import open_pickle

from cube3_game import Cube3Game
from models import Pilgrim
# from datasets import get_torch_scrambles
# from utils import set_seed

In [3]:
T = torch.tensor([
    [1, 2, 3],
    [4, 5, 6]
])

t = torch.tensor([1, 1])

print(T.shape, t.shape)

eq_res = torch.eq(T, t.unsqueeze(dim=1).expand(2, 3))
eq_res.any(dim=1)

torch.Size([2, 3]) torch.Size([2])


tensor([ True, False])

In [14]:
hash_vec = torch.tensor([1, 0, 2], dtype=torch.int64)

states = torch.tensor([
    [[1, 2, 3]],
    [[4, 5, 6]]
], dtype=torch.int64) # (N=2, n=1, STATE_SIZE=3)

exp_hash_vec = hash_vec.unsqueeze(
    dim=0
).unsqueeze(
    dim=0
).expand(
    2,
    1,
    3
)

# print("stases:", states.shape)

hashes = torch.mul(
    states, 
    exp_hash_vec
).sum(dim=2)

print("hash:", hashes)
print("hash:", hashes.shape)

hash: tensor([[ 7],
        [16]])
hash: torch.Size([2, 1])


In [4]:
A = torch.tensor([[1, 2], [2, 4]], dtype=torch.int64, device="mps")
B = torch.tensor([[1, 2], [2, 4]], dtype=torch.int64, device="mps")

torch.matmul(A.int_repr(), B.int_repr())

NotImplementedError: The operator 'aten::int_repr' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

In [44]:
torch.mul(A, B).sum(dim=1)

tensor([ 5, 20], device='mps:0')

: 

In [57]:
set_seed(0)
deepcube_test = open_pickle("../assets/data/deepcubea/data_0.pkl")
game = Cube3Game("../assets/envs/qtm_cube3.pickle")
generators = torch.tensor(game.actions, dtype=torch.int64)

# i = 0
# state = torch.tensor(deepcube_test['states'][i], dtype=torch.int64).unsqueeze(0)
# solution = deepcube_test['solutions'][i]    

states, actions, values = get_torch_scrambles(
    n = 1,
    space_size = game.space_size,
    action_size = game.action_size,
    length = 6,
    permutations = generators
)
state = states[-1, :]
value = values[-1].item()
# actions = actions[-1, :]

state = state.unsqueeze(dim=0)

model = Pilgrim(
    hidden_dim1 = 500, 
    hidden_dim2  = 300, 
    num_residual_blocks = 3
)
model.load_state_dict(torch.load("../assets/models/Cube3ResnetModel_policy.pt"))
model.eval()
pass

In [160]:
# i = 2
# state = states[i, :]
# value = values[i]
# action = actions[:i+1]

state = states[-1, :]
value = values[-1]
action = actions#[:i+1]

print("v", value)
print("actions:", action)
s = state.clone()
g = torch.arange(0, 54, dtype=torch.int64)

for a in action.tolist()[::-1]:
    ra = game.reverse_action(a)
    print(f"a: {a}={game.names[a]} -> {ra}={game.names[ra]}")  
    s = s[generators[ra]]

s_is_g = (s == g).all()
print("")

for a in action.tolist():
    # ra = game.reverse_action(a)
    # print(f"a: {a}={game.names[a]} -> {ra}={game.names[ra]}")  
    g = g[generators[a]]
    print(f"a: {a}", (g == state).all())    

print(" ")
print("s:", s, "; is_equal to goal?:", s_is_g)
print("g:", g, "; is_equal to state?: ", (g == state).all())
# print("states:", state[0, :])

v tensor(6.)
actions: tensor([8, 3, 5, 0, 7, 3])
a: 3=R -> 9=R'
a: 7=D' -> 1=D
a: 0=U -> 6=U'
a: 5=F -> 11=F'
a: 3=R -> 9=R'
a: 8=L' -> 2=L

a: 8 tensor(False)
a: 3 tensor(False)
a: 5 tensor(False)
a: 0 tensor(False)
a: 7 tensor(False)
a: 3 tensor(True)
 
s: tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]) ; is_equal to goal?: tensor(True)
g: tensor([24, 21, 18, 52,  4, 46, 35, 50, 29, 33, 30, 27, 43, 13, 37, 26,  7, 20,
        15, 23,  9, 16, 22, 10, 17, 14, 11,  2, 39,  8, 32, 31,  3,  0, 41,  6,
        47,  5, 53, 19, 40, 25, 36,  1, 42, 51, 48, 45, 34, 49, 28, 44, 12, 38]) ; is_equal to state?:  tensor(True)


In [158]:
def reverse_actions(actions: torch.Tensor, n_gens: int):
    n_gens_half = n_gens / 2
    return actions - n_gens_half * (2 * (action >= n_gens_half).int() - 1)

In [161]:
with torch.no_grad():
    _, policy = model(state)
    policy_softmax = torch.softmax(policy, dim=1)

print(policy.shape)
print(policy)
print(policy_softmax)

torch.Size([1, 12])
tensor([[-0.7971, -1.1115,  1.6928,  3.3070, -1.1702, -1.1954, -1.3794, -0.6181,
          0.0955, -0.5255, -1.2056, -1.0717]])
tensor([[0.0121, 0.0088, 0.1459, 0.7329, 0.0083, 0.0081, 0.0068, 0.0145, 0.0295,
         0.0159, 0.0080, 0.0092]])


In [162]:
action

tensor([8, 3, 5, 0, 7, 3])

In [106]:
states[0, :]

tensor([45, 46, 47,  3,  4,  5,  6,  7,  8, 44, 43, 42, 12, 13, 14, 15, 16, 17,
        20, 23, 26, 19, 22, 25, 18, 21, 24, 27, 28, 29, 30, 31, 32, 33, 34, 35,
        36, 37, 38, 39, 40, 41,  2,  1,  0,  9, 10, 11, 48, 49, 50, 51, 52, 53])

In [101]:
for i in range(12):
    g = torch.arange(0, 54, dtype=torch.int64)
    print("i:", i, ") = ", (g[generators[i]] == states[0, :]).all())

i: 0 ) =  tensor(False)
i: 1 ) =  tensor(False)
i: 2 ) =  tensor(False)
i: 3 ) =  tensor(False)
i: 4 ) =  tensor(False)
i: 5 ) =  tensor(False)
i: 6 ) =  tensor(False)
i: 7 ) =  tensor(False)
i: 8 ) =  tensor(True)
i: 9 ) =  tensor(False)
i: 10 ) =  tensor(False)
i: 11 ) =  tensor(False)
